In [ ]:
# Importing modules
import requests 
from my_api_info import get_noaa_token
from bs4 import BeautifulSoup
from time import sleep
import re

### Get list of station IDs for later

In [ ]:
stations_url = 'https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt'
r_stat = requests.get(stations_url)
r_stat.status_code

In [ ]:
stations_MI = {}

# Found this by hand
last_index = 1418
html = r_stat.text.split(' MI ',last_index+1)

# Loop through the html text and compile dict 
# with keys the GHCND station IDs, and values 
# a tuple of (latitude, longitude) of the station
for text in html:
    station_id = text[-37:-26]
    lat = text[-24:-17]
    long = text[-15:-7]
    stations_MI[station_id] = (lat,long)

# Remove last erroneous entry 
del stations_MI['GE         ']
# Remove some middle weird entries
del stations_MI['333  -81.75']
del stations_MI['  -80.8242 ']
del stations_MI['167  -81.26']
# Removes a station that's really in OH
del stations_MI['USC00201634']
# Removes stations north of Upper Peninsula 
# (in Isle Royal National Park)
del stations_MI['USC00205637']
del stations_MI['USR0000MOJI']
del stations_MI['USR0000MWDO']

In [ ]:
## Testing
for key in stations_MI.keys():
    if float(stations_MI[key][0]) < 41:
        print(key)
        print(stations_MI[key])
    elif float(stations_MI[key][0]) > 47.5:
        print(key)
        print(stations_MI[key])

In [ ]:
new_stations = {}

for s in stations_MI.keys():
    loc = stations_MI[s]
    newloc = (float(loc[0]), float(loc[1]))
    new_stations[s] = newloc
print(stations_MI)

In [ ]:
print(new_stations)

### Get list of MI ZIP codes

In [ ]:
zip_url = 'https://www.zip-codes.com/state/mi.asp'
r = requests.get(zip_url)
r.status_code

In [ ]:
soup = BeautifulSoup(r.content, 'html.parser')

In [ ]:
zip_list_helper = soup.body.find_all('a', string=re.compile('ZIP Code \d'))
MI_ZIP_list = [z.text[9:14] for z in zip_list_helper]
MI_ZIP_list[:5]

In [ ]:
len(MI_ZIP_list)

In [ ]:
# FIPS codes in MI
FIPS_MI = ['26049',
 '26115',
 '26081',
 '26163',
 '26021',
 '26005',
 '26125',
 '26155',
 '26091',
 '26161',
 '26077',
 '26059',
 '26045',
 '26145',
 '26093',
 '26067',
 '26065',
 '26099',
 '26147',
 '26139',
 '26151',
 '26037',
 '26015',
 '26157',
 '26063',
 '26159',
 '26075',
 '26087',
 '26023',
 '26129',
 '26027',
 '26017',
 '26025',
 '26007',
 '26057',
 '26073',
 '26123',
 '26133',
 '26009',
 '26041',
 '26149',
 '26111',
 '26039',
 '26069',
 '26107',
 '26103',
 '26143',
 '26051',
 '26071',
 '26035',
 '26011',
 '26001',
 '26121',
 '26165',
 '26117',
 '26079',
 '26043',
 '26113',
 '26141',
 '26003',
 '26031',
 '26119',
 '26033',
 '26109',
 '26127',
 '26047',
 '26105',
 '26137',
 '26095',
 '26053',
 '26135',
 '26085',
 '26055',
 '26131',
 '26029',
 '26089',
 '26097',
 '26019',
 '26083',
 '26153',
 '26013',
 '26101',
 '26000']

In [ ]:
print([int(f) for f in FIPS_MI])

In [ ]:
len(FIPS_MI)

In [ ]:
r = requests.get(url, headers=headers, params=parameters)

In [ ]:


r.json()['results'][0]['station']

In [ ]:
# List which stations are in which county (FIPS code)
station_FIPS = {}

headers = {'token':get_noaa_token()}

for FIPS in FIPS_MI:
    parameters = {'datasetid':'GSOM',
              'startdate':'2010-02-01',
              'enddate':'2020-02-01',
              'units':'metric',
              'datatypeid':'PRCP',
              'station':'GHCND:US',
              'locationid':'FIPS:' + FIPS}
    success = False
    r = requests.get(url,
                     headers=headers,
                     params=parameters)
    if r.status_code==200:
        success = True
    if success == True:
        results = r.json()
        if results != {}:
            station_FIPS[FIPS]=results['results'][0]['station']
        else:
            station_FIPS[FIPS]=-1
    elif success == False:
        station_FIPS[FIPS]=-1
    print(success)
    print(FIPS, station_FIPS[FIPS])
    sleep(1)

In [ ]:
# Add in the one FIPS code station manually
parameters = {'datasetid':'GSOM',
              'startdate':'2010-02-01',
              'enddate':'2020-02-01',
              'units':'metric',
              'datatypeid':'PRCP',
              'station':'GHCND:US',
              'locationid':'FIPS:' + '26085'}

r = requests.get(url,
                 headers=headers,
                 params=parameters)

In [ ]:
r.status_code

### Stations in MI, Monthly Data

In [ ]:
base_url = 'https://www.ncei.noaa.gov/cdo-web/api/v2'

In [ ]:
extension = '/data'

url = base_url + extension

In [ ]:
headers = {'token':get_noaa_token()}

In [ ]:
def check_ZIP_station(ZIP):
    parameters = {'datasetid':'GSOM',
                  'startdate':'2015-02-01',
                  'enddate':'2020-02-01',
                  'units':'metric',
                  'datatypeid':'PRCP',
                  'station':'GHCND:US',
                  'locationid':'ZIP:' + ZIP}
    r = requests.get(url,
                     headers=headers,
                     params=parameters)
    return r

In [ ]:
def check_FIP_station(FIPS):
    parameters = {'datasetid':'GSOM',
                  'startdate':'2015-02-01',
                  'enddate':'2020-02-01',
                  'units':'metric',
                  'datatypeid':'PRCP',
                  'station':'GHCND:US',
                  'locationid':'FIPS:' + FIPS}
    r = requests.get(url,
                     headers=headers,
                     params=parameters)
    return r

In [ ]:
a = check_FIP_station('26')

In [ ]:
a.json()['results'][0]['station'][6:] in stations_MI

### API interaction

In [ ]:
base_url = 'https://www.ncei.noaa.gov/cdo-web/api/v2'

In [ ]:
extension = '/data'

url = base_url + extension

In [ ]:
headers = {'token':get_noaa_token()}

In [ ]:
parameters = {'datasetid':'GSOM',
              'startdate':'2017-02-01',
              'enddate':'2017-02-01',
              'units':'metric',
              'datatypeid':'PRCP',
              'station':'GHCND:US',
              'locationid':'48104'}

In [ ]:
r = requests.get(url, 
                 headers=headers,
                 params=parameters)
r.status_code

In [ ]:
r.json()

### Interpreting Results

Calling the following line of Python code:

`requests.get(url, headers=headers, params=parameters)`

with inputs

`url = https://www.ncei.noaa.gov/cdo-web/api/v2/data`

`headers = {'token':'<NOAA_TOKEN>'}`

`parameters = {'stationid':'GHCND:USC00173046','datasetid':'GHCND','startdate':'2017-01-01','enddate':'2017-01-31','units':'metric'}`

returns a json file with the following data (truncated after one day):

`{'metadata': {'resultset': {'offset': 1, 'count': 186, 'limit': 25}},
 'results': \[{'date': '2017-01-01T00:00:00',
   'datatype': 'PRCP',
   'station': 'GHCND:USC00173046',
   'attributes': ',,7,0700',
   'value': 6.1},
  {'date': '2017-01-01T00:00:00',
   'datatype': 'SNOW',
   'station': 'GHCND:USC00173046',
   'attributes': ',,7,',
   'value': 76.0},
  {'date': '2017-01-01T00:00:00',
   'datatype': 'SNWD',
   'station': 'GHCND:USC00173046',
   'attributes': ',,7,',
   'value': 279.0},
  {'date': '2017-01-01T00:00:00',
   'datatype': 'TMAX',
   'station': 'GHCND:USC00173046',
   'attributes': ',,7,0700',
   'value': 0.6},
  {'date': '2017-01-01T00:00:00',
   'datatype': 'TMIN',
   'station': 'GHCND:USC00173046',
   'attributes': ',,7,0700',
   'value': -13.3},
  {'date': '2017-01-01T00:00:00',
   'datatype': 'TOBS',
   'station': 'GHCND:USC00173046',
   'attributes': ',,7,0700',
   'value': 0.6}\]...}`

The unitsof PRCP are in milimeters (mm). For example,the datatype 'PRCP' refers to precipitation in liquid form observed over the 24-hour observation period, and for Jan 1, 2017, at this station in Maine, it measured 6.1mm. The TMAX says the maximum temp that day was 0.6 Celsius, etc. 

### Some helpful API-accessing functions

In [ ]:
## Getting monthly rainfall given the station ID
## (a string beginning with US) and month (in the 
## form: YYYY-MM)

def get_monthly_rainfall(station_id, month):
    # Parse month correctly
    start_date = month + '-01'
    base_url = 'https://www.ncei.noaa.gov/cdo-web/api/v2'
    extension = '/data'
    headers = {'token':get_noaa_token()}
    parameters = {'stationid':'GHCND:' + station_id,
              'datasetid':'GSOM',
              'startdate':start_date,
              'enddate':start_date,
              'units':'metric',
              'datatypeid':'PRCP'}
    url = base_url + extension
    r = requests.get(url, 
                 headers=headers,
                 params=parameters)
    print('GHCND:' + station_id)
    print(r.status_code)
    print(start_date)
    if r.status_code != 200:
        return -1
    # Empty dicts evaluate to False
    if bool(r.json()):
        value = r.json()['results'][0]['value']
    else:
        value = -1
    return value

In [ ]:
get_monthly_rainfall('US1MIAN0018','2017-02')

In [ ]:
# Let's write a function that returns all station data 
# (PRCP) in Michigan for input month
# MI is FIPS:26
# Month input in YYYY-MM format
headers = {'token':get_noaa_token()}
base_url = 'https://www.ncei.noaa.gov/cdo-web/api/v2'
extension = '/data'
url = base_url + extension

def get_PRCP_MI(month):
    startdate = month + '-01'
    parameters = {'datasetid':'GSOM',
                  'startdate':startdate,
                  'enddate':startdate,
                  'units':'metric',
                  'datatypeid':'PRCP',
                  'station':'GHCND:US',
                  'locationid':'FIPS:' + '26',
                  'limit':400}
    r = requests.get(url,
                     headers=headers,
                     params=parameters)
    return r

In [ ]:
r = get_PRCP_MI('2020-01')

In [ ]:
r.json()

# Visualization

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
## First, let's just plot every station in MI

fig = plt.figure(figsize=(8,6),
                 dpi=200)

ax = fig.add_subplot()

latitudes = []
longitudes = []
for key in stations_MI.keys():
    longitudes.append(float(stations_MI[key][1]))
    latitudes.append(float(stations_MI[key][0]))
    
ax.scatter(longitudes,
           latitudes,
           label='Station',
           color='dodgerblue',
           s=40,
           alpha=0.5,
           edgecolor='white',
           linewidth=0.5)

# longitude/latitude limits of Michigan
ax.set_xlim([-90.5,-82])
ax.set_ylim([41.5,47.75])

# Niceties
ax.set_xlabel("Longitude (E/W)")
ax.set_ylabel("Latitude (N/S)")
plt.legend()
plt.title

# plt.savefig('MI_map.png',
#             format='png')

plt.show()

#### alpha of point proportional to rainfall in month

In [ ]:
# Now, let's plot every station in MI, with size equal to 
# the monthly rainfall for the month of 2023-01

# First, we need to figure out which stations have this data.
r = get_PRCP_MI('2023-01')

In [ ]:
# Here is a list of stations in this data (remember,
# rainfal is in mm)
stat_with_data = [(d['station'], d['value']) for d in r.json()['results']]
stat_with_data[:5]

In [ ]:
# Get relevant latitude/longitude
lats = []
lngs = []
for entry in stat_with_data:
    stat_id = entry[0][6:]
    entry_lat = stations_MI[stat_id][0]
    entry_lng = stations_MI[stat_id][1]
    lats.append(float(entry_lat))
    lngs.append(float(entry_lng))

In [ ]:
# Get values of rainfall for each station
rain_vals = [d[1] for d in stat_with_data]
rain_vals[:5]

In [ ]:
# Check the arrays have all the same sizes
print(len(lats)==len(lngs))
print(len(lats)==len(rain_vals))

In [ ]:
# Rescale rainfalls to be between 0 and 1 (divide
# by minimum) so they can be used for alphas
alpha_vals = np.array(rain_vals)/max(rain_vals)

In [ ]:
# Plotting 
fig = plt.figure(figsize=(8,6),
                 dpi=100)

ax = fig.add_subplot()

ax.scatter(lngs,
           lats,
           color='dodgerblue',
           s=80,
           edgecolor='white',
           linewidth=0.1,
           alpha=alpha_vals,
           label='station')


plt.show()

In [ ]:
# Another plot, with colors depending on rainfall
# Plotting 
fig = plt.figure(figsize=(8,6),
                 dpi=100)

#ax = fig.add_subplot()

plt.scatter(lngs,
           lats,
           c=alpha_vals,
           cmap='Blues',
           s=100,
           edgecolor='white',
           linewidth=0.1,
           alpha=1,
           label='station')

plt.colorbar(label='Rainfall',
             location='left')

plt.legend()
plt.show()

In [ ]:
import cartopy.crs as ccrs
import numpy as np
import cartopy.feature as cfeature
import cartopy.io.shapereader as shapereader
import pandas as pd
from metpy.cbook import get_test_data

In [ ]:
get_test_data('us_counties_20m.shp', as_file_obj=False)
get_test_data('us_counties_20m.shx', as_file_obj=False)
get_test_data('us_counties_20m.dbf', as_file_obj=False)

In [ ]:
# Make a function to return a dict of monthly rainfall in FIPS code
headers = {'token':get_noaa_token()}
base_url = 'https://www.ncei.noaa.gov/cdo-web/api/v2'
extension = '/data'
url = base_url + extension

def mo_rfl_cty(date, code):
    startdate = date + '-01'
    parameters = {'datasetid':'GSOM',
                  'startdate':startdate,
                  'enddate':startdate,
                  'units':'metric',
                  'datatypeid':'PRCP',
                  'station':'GHCND:US',
                  'locationid':'FIPS:' + code,
                  'limit':400}
    r = requests.get(url,
                     headers=headers,
                     params=parameters)
    return 

In [ ]:
mo_rfl_cty('2020-01', '26095').json()

In [ ]:
counties = shapereader.Reader(get_test_data('us_counties_20m.shp', as_file_obj=False))

In [ ]:
next(counties.records()).attributes

In [ ]:
# With only one API access call, get all MI county stations rainfall 

month = '2020-01'

headers = {'token':get_noaa_token()}
base_url = 'https://www.ncei.noaa.gov/cdo-web/api/v2'
extension = '/data'
url = base_url + extension

rfall_MI = get_PRCP_MI(month)

In [ ]:
[entry['station'] for entry in rfall_MI.json()['results']]

In [ ]:
station_FIPS.values()

In [ ]:
for station in station_FIPS.values():
    is_in = station in [entry['station'] for entry in rfall_MI.json()['results']]
    print(is_in)

In [ ]:
# Make a dictionary mapping county geometry to amount of rainfall
county_rainfall = {rec.geometry:1
                   for rec in counties.records()}

In [ ]:
def color_rain(geom):
    rfall = county_rainfall.get(geom)
    if rfall > 0: 
        cmap = plt.get_cmap('Blues')
        norm = plt.Normalize(0,500)
        facecolor = cmap(norm(rfall))
    else:
        facecolor = 'none'
    return {'edgecolor':'black', 'facecolor':facecolor}

In [ ]:
fig = plt.figure(figsize=(8,6))

ax = fig.add_subplot(projection=ccrs.LambertConformal())

ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.STATES)
ax.add_geometries(counties.geometries(),
                  ccrs.PlateCarree(),
                  styler=color_rain)
ax.set_extent((-91, -82, 41, 48))

plt.show()

In [ ]:
station_FIPS